# Plot raw fire data on map - (START)

In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import os
import pandas as pd
import numpy as np
def plot_all_occurrence():
    plt.figure(figsize=(15, 15))
    llon = -140
    ulon = -52
    llat = 41
    ulat = 75
    
    # Create a Basemap instance for Canada
    m = Basemap(projection='mill', llcrnrlat=41, urcrnrlat=83, llcrnrlon=-140, urcrnrlon=-52, resolution='c')

    # Draw the coastlines and country borders
    m.drawcoastlines()
    m.fillcontinents(color='#ddaa66',lake_color='aqua')
    #m.drawstates()
    m.drawcountries()

    # Specify the directory path
    directory = './historical_fire_data/grouped-90-percent/'

    # Loop through files in the directory
    for filename in os.listdir(directory):
        # Check if the current item is a file
        #if os.path.isfile(os.path.join(directory, filename)):
        #    print(filename)

        if filename == '.DS_Store' or filename == 'map':
            continue
    
        file_path = directory + filename
        #print(file_path)
        fire_data = pd.read_csv(file_path)
        #print(fire_data['latitude'])
    
        # Plot scatter points
        x, y = m(fire_data['longitude'], fire_data['latitude'])
        m.scatter(x, y, color='red', marker='.', zorder=5)  # zorder to plot points on top of the map


    # Draw meridians and parallels
    m.drawparallels(range(40, 90, 5), labels=[1,0,0,0], fontsize=10)
    m.drawmeridians(range(-140, -50, 5), labels=[0,0,0,1], fontsize=10)

    # Draw grid lines
    #grid_interval = 10  # 1x1 km grid
    #lon_lines = np.arange(llon, ulon, grid_interval / 111.0)  # Convert km to degrees
    #lat_lines = np.arange(llat, ulat, grid_interval / 111.0)  # Approximate 1 degree = 111 km

    #for lon in lon_lines:
    #    x, y = m([lon, lon], [llat, ulat])
    #    m.plot(x, y, color='black', linestyle='solid', linewidth=0.5)

    #for lat in lat_lines:
    #    x, y = m([llon, ulon], [lat, lat])
    #    m.plot(x, y, color='black', linestyle='solid', linewidth=0.5)

    # Add a title
    plt.title('Map of Canada - 10 years')

    #plt.savefig('plot_10_years.png')

    # Show the plot
    plt.show()

# Plot raw fire data on map - (END)

In [2]:
#import logging
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import os
import pandas as pd
import numpy as np

class RowRange:
    def __init__(self, llon, ulon):
        self.llon = llon
        self.ulon = ulon
        
class ColumnRange:
    def __init__(self, llat, ulat):
        self.llat = llat
        self.ulat = ulat

class FirePoint:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude
    def __repr__(self):
        return f"FirePoint( latitude = {self.latitude},  longitude = {self.longitude})"    

class GridCell:
    """ This class holds the coordinates - start and end of latitude and longitude """
    def __init__(self, llat, ulat, llon, ulon, fire_count):
        self.llat = llat
        self.ulat = ulat
        self.llon = llon
        self.ulon = ulon
        self.fire_count = fire_count
        self.fire_points = []
        
    def __repr__(self):
        return f"GridCell( llat = {self.llat},  ulat = {self.ulat}, llon = {self.llon}, ulon = {self.ulon}, fire_count = {self.fire_count}, fire_points_length = {len(self.fire_points)})"

class MapCoordinates:
    def __init__(self):
        self.llon = -140
        self.ulon = -55
        self.llat = 41
        self.ulat = 70
        
        self.distance = 1/111
        self.parallels = np.arange(self.llat, self.ulat, self.distance)
        self.meridians = np.arange(self.llon, self.ulon, self.distance)
        self.meridians_length = len(self.meridians) # x-axis - longitude
        self.parallels_length = len(self.parallels) # y-axis - latitude
        
class FireDataGrid:
    def __init__(self):
        map_coordinates = MapCoordinates()
        self.row_ranges = []
        self.col_ranges = []
        print(f"fire data Row {map_coordinates.meridians_length} and Columns {map_coordinates.parallels_length}")
        grid = np.full(shape = (map_coordinates.meridians_length, map_coordinates.parallels_length), fill_value = None)
        meridians_list = list(map_coordinates.meridians)
        parallels_list = list(map_coordinates.parallels)
        
        for row in range(0, map_coordinates.meridians_length):
            longitude = meridians_list[row]
            lower_longitude = longitude
            upper_longitude = longitude + map_coordinates.distance
            for col in range(0, map_coordinates.parallels_length):
                latitude = parallels_list[col]
                lower_latitude = latitude
                upper_latitude = latitude + map_coordinates.distance
                #print(type(upper_longitude))
                cell = GridCell(float(lower_latitude), 
                                float(upper_latitude), 
                                float(lower_longitude), 
                                float(upper_longitude), 0)
                #print(cell)
                grid[row, col] = cell
                if row==0 and (col == 0 or ((col+1) % 111) == 0):
                    col_range = ColumnRange(lower_latitude, upper_latitude)
                    self.col_ranges.append(col_range)
            
            if row ==0 or ((row+1) % 111) == 0:
                row_range = RowRange(lower_longitude, upper_longitude)
                self.row_ranges.append(row_range)
            if (row+1) % 111 == 0:
                print(f"finished row {row} ")
        print("Grid ready================")
        self.grid = grid

In [3]:
import pickle
def export_to_pickle(data, file_path):
    # Serialize the object and write it to a file
    print("Saving data to pickle.")
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

    print("Object exported to", file_path)
    
def load_from_pickle(file_path):
    # Deserialize the object from the file
    with open(file_path, 'rb') as f:
        data = pickle.load(f)

    # Now, loaded_obj contains the object that was stored in the file
    print("Object loaded from file:"+file_path)
    return data
def export_data_chunks(data: FireDataGrid, directory):
    row_count = 88
    index = 0
    
    while index < row_count:
        start = index * 111
        end = (index+1) * 111
        print(f"exporting data from {start} to {end}")
        sub_matrix = data.grid[index:index+111]
        export_to_pickle(sub_matrix, f"{directory}/grid_{index}_{start}_{end-1}.pickle")
        index +=1
    
    


In [4]:
def import_data_chunks(directory):
    row_count = 88
    index = 0
    map_coordinates = MapCoordinates()
    grid = np.full(shape = (0, map_coordinates.parallels_length), fill_value = None)
    while index < row_count:
        start = index * 111
        end = (index+1) * 111
        print(f"exporting data from {start} to {end}")
        
        file_name = f"{directory}/grid_{index}_{start}_{end-1}.pickle"
        sub_matrix = load_from_pickle(file_name)
        grid = np.vstack([grid,sub_matrix])
        index +=1
    return grid

In [7]:
import os
def find_next_divisible_by_10(num):
  """Finds the next integer divisible by 10.

  Args:
      num: The number to check.

  Returns:
      The next integer divisible by 10.
  """
  remainder = num % 10
  while remainder != 0:
    num +=1
    remainder = num % 10
  return num

def is_file_present( file_name):
    file_present = os.path.isfile(file_name)
    # if file_present:
    #     print('Already present : '+file_name)
    # else:
    #     print('file absent : '+file_name)
    return file_present

    
def set_fire_count(grid, row_ranges, column_ranges):
    #logging.basicConfig(filename='set_fire_count.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
    directory = './historical_fire_data/grouped-90-percent/'
    #loop over all the fire data year wise
    for filename in os.listdir(directory):
        # Check if the current item is a file
        #if os.path.isfile(os.path.join(directory, filename)):
        #    print(filename)

        #print(filename)
        if filename == '.DS_Store' or filename == 'map' or filename == 'grouped-90-percent' or filename == 'grouped':
            continue

        file_path = directory + filename
        #logging.info(f"================= {file_path}")
        print(file_path)
        fire_data = pd.read_csv(file_path)

        for index, fire_data_row in fire_data.iterrows():
            #logging.info(f"==========================================================================================================")
            confidence = fire_data_row['confidence']
            latitude = fire_data_row['latitude']
            longitude = fire_data_row['longitude']
            #loop over grid to set the fire count
            #row_str = ', '.join([f'{key}: {value}' for key, value in fire_data_row.items()])

            # Log the row
            #logging.info(f"fire data Row {index}: {row_str}")
            #logging.info(f"fire data {type(latitude)}: {type(longitude)}")
            
            lower_row_limit = 0
            upper_row_limit = 0
            lower_row_limit_found = False
            upper_row_limit_found = False
            row_check = 0
            while row_check < 86:
                if not lower_row_limit_found and row_ranges[row_check].llon > longitude:
                    lower_row_limit_found = True
                    lower_row_limit = row_check - 1
                if not upper_row_limit_found and row_ranges[row_check].ulon > longitude:
                    upper_row_limit_found = True
                    upper_row_limit = row_check
                if lower_row_limit_found and upper_row_limit_found:
                    break
                row_check +=1
            
            lower_row_limit = lower_row_limit * 111
            upper_row_limit = upper_row_limit * 111
            
            lower_col_limit = 0
            upper_col_limit = 0
            lower_col_limit_found = False
            upper_col_limit_found = False
            col_check = 0
            while col_check < 30:
                if not lower_col_limit_found and column_ranges[col_check].llat > latitude:
                    lower_col_limit_found = True
                    lower_col_limit = col_check - 1
                if not upper_col_limit_found and column_ranges[col_check].ulat > latitude:
                    upper_col_limit_found = True
                    upper_col_limit = col_check
                if lower_col_limit_found and upper_col_limit_found:
                    break
                col_check +=1
            
            lower_col_limit = lower_col_limit * 111
            upper_col_limit = upper_col_limit * 111
            
            print(f"{lower_row_limit}:{upper_row_limit} - {lower_col_limit}:{upper_col_limit}")
            
            #Assign the fire data to a grid
            found = False
            for row in range(lower_row_limit, upper_row_limit):
                for col in range(lower_col_limit, upper_col_limit):
                    grid_cell = grid[row, col]
                    
                    cell_llat = grid_cell.llat  
                    cell_ulat = grid_cell.ulat
                    cell_llon = grid_cell.llon 
                    cell_ulon = grid_cell.ulon 
                    fire_count = grid_cell.fire_count
                    fire_points = grid_cell.fire_points
                    #logging.info(f"Cell {row} {col} : {type(llat)}-{type(ulat)}-{type(llon)}-{type(ulon)}")
                    if (cell_llat <= latitude < cell_ulat) and (cell_llon <= longitude < cell_ulon):
                        found = True
                        #logging.info(f"Cell {row} {col}: {grid_cell}")
                        #print(f"Cell {row} {col}: {grid_cell}")
                        #print(found)
                        
                        fire_count = fire_count + 1
                        grid_cell.fire_count = fire_count
                        fire_points.append(FirePoint(latitude, longitude))
                        grid_cell.fire_points = fire_points
                        #logging.info(f"New Cell {row} {col}: {grid_cell}")
                        grid[row, col] = grid_cell
                        #print(f"{file_path} row completed {index} - {longitude}:{latitude} at {row}:{col} - {grid_cell}")
                        break
                        
                if found:
                    break
                        
                        
def export_grid(grid):
    grid_df = pd.DataFrame({'row_number': pd.Series(dtype='int'),
                            'lower_longitude': pd.Series(dtype='float'),
                            'upper_longitude': pd.Series(dtype='float'),
                            'lower_latitude': pd.Series(dtype='float'),
                            'upper_latitude': pd.Series(dtype='float'),
                            'fire_count': pd.Series(dtype='int')})
    map_coordinates = MapCoordinates()
    last_row = 0
    for row in range(0,map_coordinates.meridians_length):
        next_file_number = find_next_divisible_by_10(row+1)
        file_name = f"./data/counts/fire_data_grid_{next_file_number}.csv"
        
        if is_file_present(file_name):
            continue
        
        for col in range(0, map_coordinates.parallels_length):
            cell = grid[row, col]
            df_row = {
                'row_number': row,
                'lower_longitude': cell.llon,
                'upper_longitude': cell.ulon,
                'lower_latitude': cell.llat,
                'upper_latitude': cell.ulat,
                'fire_count': cell.fire_count
            }
            grid_df.loc[len(grid_df)] = df_row
        print(f"row added : {row}")
        last_row = row
        if (row+1) % 10 == 0:
            print(f"Exporting {file_name} ==================================")
            grid_df.to_csv(file_name)       
            grid_df = pd.DataFrame({'row_number': pd.Series(dtype='int'),  
                                    'lower_longitude': pd.Series(dtype='float'),
                                    'upper_longitude': pd.Series(dtype='float'),
                                    'lower_latitude': pd.Series(dtype='float'),
                                    'upper_latitude': pd.Series(dtype='float'),
                                    'fire_count': pd.Series(dtype='int')})
    grid_df.to_csv(f"./data/counts/fire_data_grid_{last_row+1}.csv")  
        
        
def plot_fire_count(grid):
    plt.figure(figsize=(15, 15))
    map_coordinates = MapCoordinates()
    # Create a Basemap instance for Canada
    m = Basemap(projection='mill', llcrnrlat=map_coordinates.llat, urcrnrlat=map_coordinates.ulat, 
        llcrnrlon=map_coordinates.llon, urcrnrlon=map_coordinates.ulon, resolution='c')
    m.drawcoastlines()
    m.fillcontinents(color='#ddaa66',lake_color='aqua')
    m.drawcountries()
    
    
    m.drawparallels(map_coordinates.parallels, labels=[1,0,0,0], fontsize=10)
    m.drawmeridians(map_coordinates.meridians, labels=[0,0,0,1], fontsize=10, rotation=90)

    for row in range(0,map_coordinates.meridians_length):
        for col in range(0, map_coordinates.parallels_length):
            cell = grid[row, col]
            latitude = (cell.llat + cell.ulat)/2.0
            longitude = (cell.llon + cell.ulon)/2.0
            x, y = m(longitude, latitude)
            if cell.fire_count >= 1:
                m.scatter(x, y, color='red', marker='.', zorder=1)  # zorder to plot points on top of the map
                #m.text(x, y, cell.fire_count, fontsize=8, ha='right', va='bottom')
                #plt.annotate(cell.fire_count, xy=(x, y), xytext=(5, 5), textcoords='offset points')

    plt.title('Map of Canada - 10 years. Y-axis - Latitude. X-axis - Longitude')

    #plt.xlabel("Longitude")
    #plt.ylabel("Latitude")
    #plt.savefig('plot_10_years.png')

    # Show the plot
    plt.show()

In [6]:
# #row_ranges1 = load_from_pickle("./data/init/row_ranges.pickle")
# #column_ranges1 = load_from_pickle("./data/init/col_ranges.pickle")
# map_coordinates = MapCoordinates()
# print(map_coordinates.parallels_length)
# print(map_coordinates.meridians_length)

3219
9435


In [8]:
fire_data_grid = FireDataGrid()

set_fire_count(fire_data_grid.grid, fire_data_grid.row_ranges, fire_data_grid.col_ranges)

fire data Row 9435 and Columns 3219
finished row 110 
finished row 221 
finished row 332 
finished row 443 
finished row 554 
finished row 665 
finished row 776 
finished row 887 
finished row 998 
finished row 1109 
finished row 1220 
finished row 1331 
finished row 1442 
finished row 1553 
finished row 1664 
finished row 1775 
finished row 1886 
finished row 1997 
finished row 2108 
finished row 2219 
finished row 2330 
finished row 2441 
finished row 2552 
finished row 2663 
finished row 2774 
finished row 2885 
finished row 2996 
finished row 3107 
finished row 3218 
finished row 3329 
finished row 3440 
finished row 3551 
finished row 3662 
finished row 3773 
finished row 3884 
finished row 3995 
finished row 4106 
finished row 4217 
finished row 4328 
finished row 4439 
finished row 4550 
finished row 4661 
finished row 4772 
finished row 4883 
finished row 4994 
finished row 5105 
finished row 5216 
finished row 5327 
finished row 5438 
finished row 5549 
finished row 5660 
fini

In [10]:
export_grid(fire_data_grid.grid)

row added : 0
row added : 1
row added : 2
row added : 3
row added : 4
row added : 5
row added : 6
row added : 7
row added : 8
row added : 9
Exporting ./data/counts/fire_data_grid_10.csv ==================================
row added : 10
row added : 11
row added : 12
row added : 13
row added : 14
row added : 15
row added : 16
row added : 17
row added : 18
row added : 19
Exporting ./data/counts/fire_data_grid_20.csv ==================================
row added : 20
row added : 21
row added : 22
row added : 23
row added : 24
row added : 25
row added : 26
row added : 27
row added : 28
row added : 29
Exporting ./data/counts/fire_data_grid_30.csv ==================================
row added : 30
row added : 31
row added : 32
row added : 33
row added : 34
row added : 35
row added : 36
row added : 37
row added : 38
row added : 39
Exporting ./data/counts/fire_data_grid_40.csv ==================================
row added : 40
row added : 41
row added : 42
row added : 43
row added : 44
row added :

KeyboardInterrupt: 